In [1]:
#Experiment denotion
EXP_DESC = {1:"DMP",2:"Sub",3:"Nov",4:"Random"}
EXP_NUM = 4     #denotes which experiment is on [1,2,3]

#Datasets
DATASET_LIST=["CAPACITIVE","OPTICAL","NIST"]
DATASET="CAPACITIVE"

#FMR denotation
FMR_DESC = {"CAPACITIVE":{1.0:35, 0.1:50, 0.01:65}, "OPTICAL":{1.0:18, 0.1:30, 0.01:40}}
FMR = 1.0    #denotes which FMR using

#Train/Test
TT_DESC=["train","full"]
TT_SET="train"   #denotes which set evaluating

#Generator
GENERATOR = None   #set it using variables later

#Classifier
#CLASS_TYPES=["Verifinger","Innovatrics","Bozorth3","MLC"]
CLASS_TYPES=["Verifinger","MLC"]
CLASSIFIER="Verifinger"


CURRENT_USERS = list(range(720))   #use defaults for now
FULL_USER_LIST = list(range(720))
NUM_USERS = 720

In [2]:
# get imports
import time
import math
import os
import sys
import random

import numpy as np
import keras
import tensorflow as tf
from keras import layers

from keras.preprocessing.image import img_to_array, ImageDataGenerator
from PIL import Image

import cma
sys.path.insert(1, 'PATH/TO/VERIFINGER/SDK/') #insert path to Verifinger SDK
import sub_verifinger as sv

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline


os.environ["CUDA_VISIBLE_DEVICES"]="-1"   #define gpu

In [3]:
GENERATOR = None

#import the variational autoencoder
def import_VAE(dataset,vae_size):
    return keras.models.load_model(f"autoencoder_models/print_{dataset}_var_decoder-{vae_size}.h5")     #change this to the correct path

#create a single sample using the generator given an input latent vector
def generateSample(x):
    xs = np.array(x)
    if(len(xs.shape)==1):   #assume 1d
        xs = np.expand_dims(xs,axis=0)
        
    return np.array(GENERATOR(xs,training=False)[0])

In [4]:
#import the contents of a txt file to an archive dictionary
def importArchiveTxt(fpath):
    all_arcs = []
    temp_arc = {}
    
    #open, read, and close file
    f = open(fpath, 'r')
    entries = f.readlines()
    f.close()
    
    #add each line as an entry to the archive
    for e in entries:
        if "~ ~ ~" in e:
            all_arcs.append(temp_arc)
            temp_arc={}
            continue
            
        ep = e.split(":")
        binnum = ep[0]
        x = ep[1]
        
        temp_arc[binnum]=[float(i) for i in x.split(",")]
        
    return all_arcs

In [5]:
#converts the user list to a binary value for storage within the archive
def userlist2Bin(classUserList,numUsers):
    bstr = np.zeros(numUsers)
    for u in classUserList:
        nu = u-min(FULL_USER_LIST)
        if u in FULL_USER_LIST:
            bstr[nu] = 1
    return "".join([str(int(i)) for i in bstr])

#convert a binary number to the user list
def bin2Userlist(bstr):
    ulist=[]
    for i,b in enumerate(bstr):
        if int(b) == 1:
            ulist.append(i+min(FULL_USER_LIST))
    return ulist


In [6]:
import sub_verifinger as sv  #import verifinger wrapper
CLASSIFIER_MODEL=None               #set MLC model to call later

#import multi classifier model
def setMLC(dataset):
    if dataset=='CAPACITIVE':
        return keras.models.load_model('multiclass_models/print_multiclassifier_720.h5')     #change this to the correct path
    #add other models here
    
#get user classification from a specific model type
#returns an array of ints corresponding to the user id (starting from 0)
def getUserPredictions(img,model,dataset,threshold,t_set='full',name='temp'):
    global CLASSIFIER_MODEL
    #check the input image formatting
    img_m=np.array(img)
    if len(img_m.shape) == 2:   # assume shape is [#,#]
        img_m=np.expand_dims(img_m,axis=2)
    if len(img_m.shape) == 3:   # assume shape is [#,#,1]
        img_m=np.expand_dims(img_m,axis=0)
    
    #get predictions straight from the mlc model
    if model=='MLC':
        thresh_p = threshold/100.0
        pred = CLASSIFIER_MODEL.predict(img_m)[0]
        users = np.where(pred >= thresh_p)[0]
        return users
    
    #verifinger calls a wrapper to get user results
    elif model=='Verifinger':
        users = sv.usersMatched(img_m, dataset.lower(),threshold,name,t_set)
        return [int(u) for u in users]
        
    #no other model found
    else:
        print(f"No model [ {model} ] found")
        return []

In [7]:
#calculate the coverage of users by the archive
def calcArcCoverage(arc,numUsers):
    usersCovered = np.zeros(numUsers)
    for c in arc.keys():
        b = [int(z) for z in c]   #convert to bitwise array (assume len==numUsers)
        
        #bitwise operation rewrite
        for i in range(numUsers):
            usersCovered[i] = (int(usersCovered[i]) | b[i])
    
    #return percentage of users found
    return sum(usersCovered)/numUsers
        

#generate a report for the experiment based on the coverage and parameters
def archiveCoverageReport(coverArr,fname):
    #save parameters
    rep = {}
    rep["EXPERIMENT"] = EXP_DESC[EXP_NUM]
    rep["DATASET"] = DATASET
    rep["FMR"] = FMR
    rep["TTYPE"] = TT_SET
    rep["CLASSIFIER"] = CLASSIFIER
    
    #save full coverage array (sorted from greatest to least)
    rep["COVERAGE"] = sorted(coverArr,reverse=True)
    
    #print to a file
    with open(fname, 'a+') as f:
        for k,v in rep.items():
            f.write(f"{k}:{v}\n")
        f.write("\n\n")
    

In [8]:
arc_date = ""
def runTestCov(archive_path):
    global CURRENT_USERS
    global FULL_USER_LIST
    global NUM_USERS
    
    # -- DEEP MASTERPRINTS EXPERIMENT -- #
    if EXP_NUM == 1:
        
        #set number of users and initial userset
        NUM_USERS = 360
    
        CURRENT_USERS = list(range(360,720))   
        FULL_USER_LIST = list(range(360,720))
        
        #intialize coverage array
        COVERAGE_ARR = []
         
        #IMPORT THE ARCHIVE LIST WITH Z VECTORS
        ARCHIVE_SET = importArchiveTxt(archive_path)
        print(len(ARCHIVE_SET))
        print(len(ARCHIVE_SET[0]))
        
        t=0
        comb_users = None
        for a in ARCHIVE_SET:
            print(f"---  TRIAL #{t} ---")
            
            ARCHIVE = a
            
            for k,z in ARCHIVE.items():   #should only be one
                #get user list and binary number
                fmr_val = FMR_DESC[DATASET][FMR]
                print_name = f"print_{DATASET}_{fmr_val}-exp_{EXP_NUM}"
                gen_print = generateSample(z)
                usersFound = getUserPredictions(gen_print,CLASSIFIER,DATASET,fmr_val,TT_SET,print_name)

                user_binstr = userlist2Bin(usersFound,NUM_USERS)
                
                if not comb_users:       # comb user is user id's found in all prints
                    comb_users = int(user_binstr,2)
                else:
                    comb_users = int(user_binstr,2) | comb_users
                
                print(f"users found: {usersFound} | bin: {user_binstr}")
                
                fake_arc={user_binstr:z}

                #save coverage (single item in archive)
                cov = calcArcCoverage(fake_arc,NUM_USERS)
                print(f"Coverage: {cov}")
                COVERAGE_ARR.append(cov)

            t+=1
            
        print(f"COMBINED USERS: {comb_users}")
        print(f"COMBINED USERS #: {str(bin(comb_users)).count('1')}")
        
        
        #export report
#         cov_d="sp-exp_coverage-out/"
#         if not os.path.exists(cov_d):
#             os.mkdir(cov_d)
#         cov_report_filename = f"[SPEXP-coverage]_report_EXP-{EXP_NUM}_{arc_date}.txt"
#         archiveCoverageReport(COVERAGE_ARR,os.path.join(cov_d,cov_report_filename))
            
            
    # -- SUBPRINT EXPERIMENT -- #
    elif EXP_NUM == 2:
        #intialize coverage array
        COVERAGE_ARR = []
        
        #IMPORT THE ARCHIVE LIST OF Z VECTORS
        ARCHIVE_SET = importArchiveTxt(archive_path)
        print(f"TRIALS: {len(ARCHIVE_SET)}")
        print(f"Members: {len(ARCHIVE_SET[0])}")
        
        t = 0
        for a in ARCHIVE_SET:
        
            print(f"---------     TRIAL #{t+1}    ----------")
            
            ARCHIVE = a

            #set number of users and initial userset
            NUM_USERS = 360    
            CURRENT_USERS = list(range(360,720))   
            FULL_USER_LIST = list(range(360,720))     
            
           
            NEW_ARCHIVE = {}
            t2=0
            for k,z in ARCHIVE.items():
                print(f"*** Archive iteration {t2+1} / {len(ARCHIVE)} ***")

                #get user list and binary number
                fmr_val = FMR_DESC[DATASET][FMR]
                print_name = f"print_{DATASET}_{fmr_val}-exp_{EXP_NUM}"
                gen_print = generateSample(z)
                usersFound = getUserPredictions(gen_print,CLASSIFIER,DATASET,fmr_val,TT_SET,print_name)

                #SUBPRINTS - REMOVE USERS FOUND
                for u in usersFound:
                    if u in CURRENT_USERS:
                        CURRENT_USERS.remove(u)

                #add to lv and classification bin string to archive
                user_binstr = userlist2Bin(usersFound,NUM_USERS)
                NEW_ARCHIVE[user_binstr] = z
                t2+=1
            t+=1

            #save coverage (single item in archive)
            cov = calcArcCoverage(NEW_ARCHIVE,NUM_USERS)
            print(f"Coverage: {cov}")
            COVERAGE_ARR.append(cov)
            
        
        #export report
#         cov_d="sp-exp_coverage-out/"
#         if not os.path.exists(cov_d):
#             os.mkdir(cov_d)
#         cov_report_filename = f"[SPEXP-coverage]_report_EXP-{EXP_NUM}_{arc_date}.txt"
#         archiveCoverageReport(COVERAGE_ARR,os.path.join(cov_d,cov_report_filename))
        
        
        
    # -- NOVELTY EXPERIMENT -- #
    elif EXP_NUM == 3:
        #intialize coverage array
        COVERAGE_ARR = []
        
        #IMPORT THE ARCHIVE LIST OF Z VECTORS
        ARCHIVE_SET = importArchiveTxt(archive_path)
        print(f"TRIALS: {len(ARCHIVE_SET)}")
        print(f"Members: {len(ARCHIVE_SET[0])}")
        
        t = 0
        for a in ARCHIVE_SET:
        
            print(f"---------     TRIAL #{t+1}    ----------")
            
            ARCHIVE = a

            #set number of users and initial userset
            NUM_USERS = 360    
            CURRENT_USERS = list(range(360,720))   
            FULL_USER_LIST = list(range(360,720))    
            
            
            NEW_ARCHIVE = {}
            t2=0
            for k,z in ARCHIVE.items():
                print(f"*** Archive iteration {t2+1} / {len(ARCHIVE)} ***")

                #get user list and binary number
                fmr_val = FMR_DESC[DATASET][FMR]
                print_name = f"print_{DATASET}_{fmr_val}-exp_{EXP_NUM}"
                gen_print = generateSample(z)
                usersFound = getUserPredictions(gen_print,CLASSIFIER,DATASET,fmr_val,TT_SET,print_name)

                #add to lv and classification bin string to archive
                user_binstr = userlist2Bin(usersFound,NUM_USERS)
                NEW_ARCHIVE[user_binstr] = z
                t2+=1
            t+=1

            #save coverage (single item in archive)
            cov = calcArcCoverage(NEW_ARCHIVE,NUM_USERS)
            print(f"Coverage: {cov}")
            COVERAGE_ARR.append(cov)
        
        #export report
        # cov_d="sp-exp_coverage-out/"
        # if not os.path.exists(cov_d):
        #     os.mkdir(cov_d)
        # cov_report_filename = f"[SPEXP-coverage]_report_EXP-{EXP_NUM}_{arc_date}.txt"
        # archiveCoverageReport(COVERAGE_ARR,os.path.join(cov_d,cov_report_filename))
        
        
    # -- RANDOM ARCHIVE EXPERIMENT -- #
    elif EXP_NUM == 4:
        #intialize coverage array
        COVERAGE_ARR = []
        
        #IMPORT THE ARCHIVE LIST OF Z VECTORS
        ARCHIVE_SET = importArchiveTxt(archive_path)
        print(f"TRIALS: {len(ARCHIVE_SET)}")
        print(f"Members: {len(ARCHIVE_SET[0])}")
        
        t = 0
        for a in ARCHIVE_SET:
        
            print(f"---------     TRIAL #{t+1}    ----------")
            
            ARCHIVE = a

            #set number of users and initial userset
            NUM_USERS = 360  
            CURRENT_USERS = list(range(360,720))   
            FULL_USER_LIST = list(range(360,720))    
            
            
            NEW_ARCHIVE = {}
            t2=0
            for k,z in ARCHIVE.items():
                print(f"*** Archive iteration {t2+1} / {len(ARCHIVE)} ***")

                #get user list and binary number
                fmr_val = FMR_DESC[DATASET][FMR]
                print_name = f"print_{DATASET}_{fmr_val}-exp_{EXP_NUM}"
                gen_print = generateSample(z)
                usersFound = getUserPredictions(gen_print,CLASSIFIER,DATASET,fmr_val,TT_SET,print_name)

                #add to lv and classification bin string to archive
                user_binstr = userlist2Bin(usersFound,NUM_USERS)
                NEW_ARCHIVE[user_binstr] = z
                t2+=1
            t+=1

            #save coverage (single item in archive)
            cov = calcArcCoverage(NEW_ARCHIVE,NUM_USERS)
            print(f"Coverage: {cov}")
            COVERAGE_ARR.append(cov)
        
        # #export report
        # cov_d="sp-exp_coverage-out/"
        # if not os.path.exists(cov_d):
        #     os.mkdir(cov_d)
        # cov_report_filename = f"[SPEXP-coverage]_report_EXP-{EXP_NUM}_{arc_date}.txt"
        # archiveCoverageReport(COVERAGE_ARR,os.path.join(cov_d,cov_report_filename))
        

In [9]:
#get the archive file by a string match
def getMatchArchiveFile(files, mstr):
    for f in files:
        if mstr in f:
            return f
    return None   #no match found

In [ ]:
#Experiment denotion
#EXPs = [1,2,3,4]
EXPs = [1]
EXP_NUM = 1     #denotes which experiment is on [1,2,3,4]

#Datasets
DATASETs = ["CAPACITIVE"]
DATASET="CAPACITIVE"

#FMR denotation
FMRs = [1.0, 0.1, 0.01]
FMR = 1.0    #denotes which FMR using

#TEST ONLY
TTs = ["test"]
TT_SET="test"   #denotes which set evaluating

#Classifier
CLASSs=["Verifinger"]
CLASSIFIER="Verifinger"

arc_date = "26-Jan-2022"

#do the thing
for c in CLASSs:
    for d in DATASETs:
        if c == "MLC":
            CLASSIFIER_MODEL = setMLC(d)
        for t in TTs:
            for e in EXPs:
                for f in FMRs:
                    
                    #set parameters
                    EXP_NUM = e
                    FMR = f
                    TT_SET = t
                    DATASET = d
                    CLASSIFIER = c
                    
                    print(f"-- EXPERIMENT: {EXP_NUM}-- ")
                    print(f"FMR: {FMR}")
                    print(f"T SET: {TT_SET}")
                    print(f"DATASET: {DATASET}")
                    print(f"CLASSIFIER: {CLASSIFIER}")
                    print("")
                    
                    #set archive path
                    arc_dir_path = f"sp-exp_archive-out/[EXP {EXP_NUM}] ({arc_date})"
                    arc_files = [os.path.join(arc_dir_path, f) for f in os.listdir(arc_dir_path) if os.path.isfile(os.path.join(arc_dir_path, f))]
                    arc_fmr_file = getMatchArchiveFile(arc_files,f"fmr-{FMR}")
                    
                    #set generator
                    GENERATOR = import_VAE(DATASET,32)
                    
                    #run it!
                    runTestCov(arc_fmr_file)
